<a href="https://colab.research.google.com/github/Narabcs/doctorship/blob/main/fcp_nara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.optimize import differential_evolution as de
from scipy.integrate import quad
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
git clone https://github.com/Narabcs/doctorship/

### Etapa 01:

Calculando os valores de tortuosidade utilizando os dados da água
---

In [3]:
dados = pd.read_csv('/content/doctorship/parameters_water.csv')
dados['y_ress'] = dados['yexp']/dados['xexp']
dados.head()

,yexp,mi,xexp,pf,meio,y_ress
0,31500000.0,0.000891,0.0080,997.05,50,3.937500e+09
1,63000000.0,0.000891,0.0125,997.05,50,5.040000e+09
2,94500000.0,0.000891,0.0148,997.05,50,6.385135e+09
3,126000000.0,0.000891,0.0173,997.05,50,7.283237e+09
4,157000000.0,0.000891,0.0193,997.05,50,8.134715e+09


In [4]:
ress_50 = stats.linregress(x=dados.query('meio==50')['xexp'],y=dados.query('meio==50')['y_ress'])
ress_120 = stats.linregress(x=dados.query('meio==120')['xexp'],y=dados.query('meio==120')['y_ress'])

results = {'meio=50':{'r_squared':ress_50.rvalue,
           'intercept': ress_50.intercept,
           'slope': ress_50.slope},
           'meio=120':{'r_squared':ress_120.rvalue,
           'intercept': ress_120.intercept,
           'slope': ress_120.slope}
           }

results = pd.DataFrame(results)
results

,meio=50,meio=120
r_squared,9.905125e-01,9.960379e-01
intercept,1.680439e+08,2.275211e+08
slope,4.229806e+11,2.715969e+11


In [5]:
def tortuosity(slope,k,pf):
  c = slope*k**0.5/pf

  return c

In [6]:
c_50 = tortuosity(results['meio=50'].slope,1.48e-11,dados['pf'].unique())
c_120 = tortuosity(results['meio=120'].slope,3.95e-11,dados['pf'].unique())
print(f'c_50 = {float(c_50)}, c_120 = {float(c_120)}')

c_50 = 1632.0532436779447, c_120 = 1712.010470506321


### Etapa 02: 

Estimando os valores de K para as equações de Forchheimmer e Darcy.

In [7]:
dados_estimacao = pd.read_csv('/content/doctorship/parameters_onlygx.csv')
dados_estimacao.head()

,yexp,m,n,xexp,es,pf,gx,meio,c
0,31500000.0,0.774,0.326,0.00306,0.354,998.28,0.2,50,1650.0
1,63000000.0,0.784,0.321,0.00703,0.354,998.28,0.2,50,1650.0
2,94500000.0,0.762,0.325,0.01030,0.354,998.28,0.2,50,1650.0
3,126000000.0,0.767,0.322,0.01180,0.354,998.28,0.2,50,1650.0
4,157000000.0,0.771,0.317,0.01450,0.354,998.28,0.2,50,1650.0


In [8]:
def forch_opt(x, dados):
  forch = (dados[1] * ((1.1/(2.5*dados[4])**0.5) * dados[3]/x[0]**0.5)**(dados[2]-1))/x[0] * dados[3] + dados[6]*dados[5]/x[0]**0.5 * dados[3]**2
  err_rel = abs(dados[0] - forch) / dados[0] 

  return err_rel

def darcy_opt(k, dados):
  darcy = (dados[1] * ((1.1/(2.5*dados[4])**0.5) * dados[3]/k**0.5)**(dados[2]-1))/k * dados[3]
  err_rel = abs(dados[0] - darcy) / dados[0] 
  return err_rel

def modelo(dados, model:str='forch'):
  
  if model == 'forch':
    mu = (dados['m'] * ((1.1/(2.5*dados['es'])**0.5) * dados['xexp']/dados['k_forch']**0.5)**(dados['n']-1))
    y = mu/dados['k_forch'] * dados['xexp'] + dados['c']*dados['pf']/dados['k_forch']**0.5 * dados['xexp']**2

  if model == 'darcy':
    mu = (dados['m'] * ((1.1/(2.5*dados['es'])**0.5) * dados['xexp']/dados['k_darcy']**0.5)**(dados['n']-1))
    y = mu/dados['k_darcy'] * dados['xexp']

  return y, mu

In [9]:
params_bound = [(1e-20,1e-10)]

results = []

for row in dados_estimacao.iterrows():
  dados_selecionados = [row[1]['yexp'], row[1]['m'], row[1]['n'], row[1]['xexp'], row[1]['es'], row[1]['pf'], row[1]['c']]
  value = de(func=forch_opt,bounds=params_bound,args=(dados_selecionados,))
  results.append(value)

df_forch = pd.DataFrame(results)

k_bound = [(1e-20,1e-10)]

results = []

for row in dados_estimacao.iterrows():
  dados_selecionados = [row[1]['yexp'], row[1]['m'], row[1]['n'], row[1]['xexp'], row[1]['es'], row[1]['pf'], row[1]['c']]
  value = de(func=darcy_opt,bounds=k_bound,args=(dados_selecionados,))
  results.append(value)

df_darcy = pd.DataFrame(results)

In [10]:
k_forch = []
k_darcy = []

for item in zip(df_forch.x,df_darcy.x):
  k_forch.append(item[0][0])
  k_darcy.append(item[1][0])
dados_estimacao['k_forch'] = k_forch
dados_estimacao['k_darcy'] = k_darcy
dados_estimacao['dpl_forch'], dados_estimacao['mu_forch'] = modelo(dados_estimacao,model='forch')
dados_estimacao['dpl_darcy'], dados_estimacao['mu_darcy'] = modelo(dados_estimacao,model='darcy')
dados_estimacao.head()

,yexp,m,n,xexp,es,pf,gx,meio,c,k_forch,k_darcy,dpl_forch,mu_forch,dpl_darcy,mu_darcy
0,31500000.0,0.774,0.326,0.00306,0.354,998.28,0.2,50,1650.0,6.616216e-13,1.102050e-13,31500000.0,0.002711,4.114602e+07,0.001482
1,63000000.0,0.784,0.321,0.00703,0.354,998.28,0.2,50,1650.0,2.138180e-12,8.233105e-14,63000000.0,0.002229,6.300000e+07,0.000738
2,94500000.0,0.762,0.325,0.01030,0.354,998.28,0.2,50,1650.0,3.867145e-12,5.487746e-14,94500000.0,0.002117,9.450000e+07,0.000503
3,126000000.0,0.767,0.322,0.01180,0.354,998.28,0.2,50,1650.0,3.652774e-12,3.648027e-14,126000000.0,0.001857,1.260000e+08,0.000390
4,157000000.0,0.771,0.317,0.01450,0.354,998.28,0.2,50,1650.0,5.183517e-12,2.670566e-14,157000000.0,0.001748,1.570000e+08,0.000289


In [11]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [14]:
# with open('/gdrive/My Drive/foo.txt', 'w') as f:
#   f.write('Hello Google Drive!')
# !cat '/gdrive/My Drive/foo.txt'

dados_estimacao.to_csv('/gdrive/MyDrive/Colab Notebooks/etapa_3.csv')

### Etapa 03:

Estimar a permeabilidade da torta

In [55]:
def fo_eval(x,params):

  def diff_model(t,params,beta):
    # beta = [bec, bkc, bkm]

    dP = params['dP']
    m = params['m']
    n = params['n']
    # esc = params['esc']
    pfil = params['pfil']
    ps = params['ps']
    A = params['A']
    Cs = params['Cs']
    lm = params['lm']
    km0 = params['km0']

    ec = np.exp(-beta[0]*t)
    kc = np.exp(beta[1]/t) - 1
    km = km0*np.exp(-beta[2]*t)
    alpha = 1 / (ps*(1-ec)*kc)
    b_aux = m*(1.1/(2.5*ec)**0.5 * 1/ kc**0.5)**(n-1)

    y = A*(dP/(b_aux*(alpha*pfil*Cs/A + lm/km)))**(1/n)
    
    return y

  beta = [x[0], x[1], x[2]]
  dV_cal = []
  
  for row in params.iterrows():
    exp_line = dict(row[1])
    t = exp_line['t']
    t0 = exp_line['t0']
    int_result = quad(diff_model, t, t0, args=(exp_line,beta))
    dV_cal.append(int_result[0])

  # print(dV_cal)
  params['dV_cal'] = dV_cal

  rel_err_for_row = abs(params['dV_real']-params['dV_cal']) / params['dV_real']
  mean_rel_err = rel_err_for_row.mean()

  return mean_rel_err

In [30]:
dados_torta = pd.read_csv('/content/doctorship/parameters_gx_caco3.csv')
dados_torta['dV_real'] = dados_torta['V']-dados_torta['V0']
dados_torta.head()

,dP,m,n,esc,pfil,ps,A,Cs,t,t0,V,V0,lm,km0,gx,meio,dV_real
0,200000.0,0.81,0.30,0.510,1084.941911,2808,0.00229,0.000133,1800,602.934,0.000017,0.000004,0.00635,1.480000e-11,0.2,50,0.000012
1,400000.0,0.69,0.33,0.544,1067.154343,2808,0.00229,0.000133,1800,599.136,0.000037,0.000008,0.00635,1.480000e-11,0.2,50,0.000028
2,600000.0,0.74,0.31,0.516,1084.007172,2808,0.00229,0.000133,1800,599.136,0.000043,0.000005,0.00635,1.480000e-11,0.2,50,0.000038
3,800000.0,0.63,0.35,0.527,1051.374826,2808,0.00229,0.000133,1800,301.056,0.000045,0.000009,0.00635,1.480000e-11,0.2,50,0.000036
4,1000000.0,0.60,0.37,0.524,1061.390808,2808,0.00229,0.000133,1800,159.414,0.000047,0.000008,0.00635,1.480000e-11,0.2,50,0.000039


In [31]:
data_selection = dados_torta.query('gx==0.2 and meio==50 and t0>0.0')
data_selection

,dP,m,n,esc,pfil,ps,A,Cs,t,t0,V,V0,lm,km0,gx,meio,dV_real
0,200000.0,0.81,0.30,0.510,1084.941911,2808,0.00229,0.000133,1800,602.934,0.000017,0.000004,0.00635,1.480000e-11,0.2,50,0.000012
1,400000.0,0.69,0.33,0.544,1067.154343,2808,0.00229,0.000133,1800,599.136,0.000037,0.000008,0.00635,1.480000e-11,0.2,50,0.000028
2,600000.0,0.74,0.31,0.516,1084.007172,2808,0.00229,0.000133,1800,599.136,0.000043,0.000005,0.00635,1.480000e-11,0.2,50,0.000038
3,800000.0,0.63,0.35,0.527,1051.374826,2808,0.00229,0.000133,1800,301.056,0.000045,0.000009,0.00635,1.480000e-11,0.2,50,0.000036
4,1000000.0,0.60,0.37,0.524,1061.390808,2808,0.00229,0.000133,1800,159.414,0.000047,0.000008,0.00635,1.480000e-11,0.2,50,0.000039
5,1200000.0,0.62,0.34,0.515,1046.896458,2808,0.00229,0.000133,1800,295.704,0.000050,0.000008,0.00635,1.480000e-11,0.2,50,0.000041
6,1400000.0,0.63,0.34,0.551,1039.610998,2808,0.00229,0.000133,1800,60.000,0.000061,0.000010,0.00635,1.480000e-11,0.2,50,0.000051


In [56]:
params_bound = [(-100,100), (-100,100), (-100,100)]
# params_bound = [bec, bkc, bkm]

results = []
value = de(func=fo_eval, bounds=params_bound, args=(data_selection,), maxiter=5000)
results.append(value)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launche

,x,fun,nfev,nit,message,success
0,[1.1020501047771157e-13],3.062230e-01,167,10,Optimization terminated successfully.,True
1,[8.233104602766657e-14],7.568844e-15,707,46,Optimization terminated successfully.,True
2,[5.4877464958991486e-14],2.302190e-14,902,59,Optimization terminated successfully.,True
3,[3.6480268263155556e-14],2.176043e-14,737,48,Optimization terminated successfully.,True
4,[2.6705662059332367e-14],4.954399e-14,812,53,Optimization terminated successfully.,True


In [58]:
df_torta = pd.DataFrame(results)
df_torta

,x,fun,nfev,nit,message,success
24,[4.575036520681738e-14],4.139211e-15,842,55,Optimization terminated successfully.,True
25,[3.262649922788469e-14],1.385713e-14,842,55,Optimization terminated successfully.,True
26,[2.5860925009680378e-14],2.996001e-14,737,48,Optimization terminated successfully.,True
27,[2.086311725184137e-14],2.953139e-14,797,52,Optimization terminated successfully.,True
28,"[-29.70505017703117, -32.22100124175466, 48.62...",NaN,45129,1000,Maximum number of iterations has been exceeded.,False


In [ ]:
k_torta = []

for item in df_torta.x:
  k_torta.append(float(item))
  
dados_torta['k_torta'] = k_torta
dados_torta.head()

,yexp,m,n,xexp,es,esc,pfil,ps,A,Cs,t,t0,V,V0,Rm,gx,meio,dt_real,k_torta
0,31500000.0,0.81,0.30,0.000004,0.354,0.510,1084.941911,2808,0.00229,0.000133,1800,602.934,0.000017,0.000004,429000000.0,0.2,50,1197.066,9.940013e-11
1,63000000.0,0.69,0.33,0.000009,0.354,0.544,1067.154343,2808,0.00229,0.000133,1800,599.136,0.000037,0.000008,429000000.0,0.2,50,1200.864,9.353011e-11
2,94500000.0,0.74,0.31,0.000010,0.354,0.516,1084.007172,2808,0.00229,0.000133,1800,599.136,0.000043,0.000005,429000000.0,0.2,50,1200.864,9.987473e-11
3,126000000.0,0.63,0.35,0.000011,0.354,0.527,1051.374826,2808,0.00229,0.000133,1800,301.056,0.000045,0.000009,429000000.0,0.2,50,1498.944,9.978466e-11
4,157000000.0,0.60,0.37,0.000011,0.354,0.524,1061.390808,2808,0.00229,0.000133,1800,159.414,0.000047,0.000008,429000000.0,0.2,50,1640.586,9.753679e-11
